In [ ]:
!pip install tensorflow==2.15.0 
!pip install keras-ocr==0.9.3 
!pip install keras==2.15.0

In [3]:
import keras_ocr
import cv2
import math
import numpy as np
from PIL import Image
import os
pipeline = keras_ocr.pipeline.Pipeline()


2024-04-21 03:56:51.648393: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 03:56:51.648535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 03:56:51.945664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [8]:

def mid(x1, y1, x2, y2):
    xm = int((x1 + x2)/2)
    ym = int((y1 + y2)/2)
    return (xm, ym)


def remove_text(img_path,out_dir):
    img = keras_ocr.tools.read_and_fit(img_path,300,300) 
    preds = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in preds[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = mid(x1, y1, x2, y2)
        x_mid1, y_mid1 = mid(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255,    
        thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
    im = Image.fromarray(img)
    out="filtered_"+os.path.splitext(os.path.basename(img_path))[0]+os.path.splitext(os.path.basename(img_path))[1]
    output_path=os.path.join(out_dir, out)
    #print(output_path)
    im.save(output_path)
    
                 
    return(img)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

IMG_DIR="/kaggle/input/facebook-hateful-meme-dataset/data/img/"
OUT_DIR="/kaggle/working/filtered_imgs/"

from pathlib import Path
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

c=0

for dirname, _, filenames in os.walk(IMG_DIR):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        if c%100==0:print(c)
        remove_text(os.path.join(dirname, filename),OUT_DIR)
        c+=1

In [10]:
import shutil 
import os.path


# Creating the ZIP file 
archived = shutil.make_archive('filtered_images', 'zip', '/kaggle/working/filtered_imgs')